### 1. Import thư viện cần thiết

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

### 2. Khởi tạo Spark

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("IE212_Lab2_Bai3") \
    .getOrCreate()
sc = spark.sparkContext

your 131072x1 screen size is bogus. expect trouble
25/11/25 21:56:39 WARN Utils: Your hostname, ELUCIDATOR resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/25 21:56:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/25 21:56:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/25 21:56:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/25 21:56:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


### 3. Load dữ liệu

In [3]:
movies_path = "data/movies.txt"
ratings1_path = "data/ratings_1.txt"
ratings2_path = "data/ratings_2.txt"
users_path = "data/users.txt"

In [4]:
movies_rdd = sc.textFile(movies_path)

In [5]:
ratings_rdd = sc.textFile(f"{ratings1_path},{ratings2_path}")

In [6]:
users_rdd = sc.textFile(users_path)

### 4. Giải quyết bài toán

In [7]:
# 2. Xử lý Users: Lấy UserID và Gender
# Input: "1::F::1::10::48067" (Giả sử file users phân cách bởi "::" hoặc ",", cần kiểm tra format thực tế)
# Lưu ý: Theo đề bài Lab, schema là UserID, Gender... Bạn cần check kỹ ký tự phân cách trong file thật.
# Ở đây tôi giả định phân cách là "," cho thống nhất với các bài trước.
def parse_user_gender(line):
    parts = line.split(",") # Hoặc "::" tùy dữ liệu thực tế
    user_id = parts[0]
    gender = parts[1] # 'M' hoặc 'F'
    return (user_id, gender)

In [8]:
# 3. Xử lý Ratings: Lấy UserID làm Key để Join
# Output: (UserID, (MovieID, Rating))
def parse_rating_for_join(line):
    parts = line.split(",")
    user_id = parts[0]
    movie_id = parts[1]
    rating = float(parts[2])
    return (user_id, (movie_id, rating))

In [9]:
user_gender_rdd = users_rdd.map(parse_user_gender)
ratings_user_key = ratings_rdd.map(parse_rating_for_join)

In [10]:
# 4. Join Users và Ratings
# user_gender_rdd: (UserID, Gender)
# ratings_user_key: (UserID, (MovieID, Rating))
# Kết quả Join: (UserID, (Gender, (MovieID, Rating)))
joined_user_rating = user_gender_rdd.join(ratings_user_key)

In [11]:
# 5. Chuyển đổi cấu trúc để tính toán theo Phim
# Mục tiêu: Chuyển về Key là MovieID để reduce
# Output: (MovieID, (SumMale, CountMale, SumFemale, CountFemale))
def map_to_movie_gender_stats(record):
    # record: (UserID, (Gender, (MovieID, Rating)))
    data = record[1]
    gender = data[0]
    movie_data = data[1]
    movie_id = movie_data[0]
    rating = movie_data[1]
    
    # Khởi tạo vector thống kê: (SumM, CountM, SumF, CountF)
    if gender == 'M':
        return (movie_id, (rating, 1, 0.0, 0))
    else:
        return (movie_id, (0.0, 0, rating, 1))

# Reduce: Cộng dồn các vector thống kê
movie_gender_stats = joined_user_rating.map(map_to_movie_gender_stats) \
    .reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1], a[2]+b[2], a[3]+b[3]))

In [12]:
# 6. Lấy tên phim (Map MovieID -> Title)
def parse_movie_title(line):
    parts = line.split(",")
    return (parts[0], parts[1])

movies_info = movies_rdd.map(parse_movie_title)

In [13]:
# 7. Join thống kê với tên phim và tính trung bình
# movie_gender_stats: (MovieID, (SumM, CntM, SumF, CntF))
# movies_info:        (MovieID, Title)
final_data = movie_gender_stats.join(movies_info)

In [14]:
# Hàm định dạng kết quả cuối cùng
def format_final_result(record):
    # record: (MovieID, ((SumM, CntM, SumF, CntF), Title))
    stats = record[1][0]
    title = record[1][1]
    
    sum_m, cnt_m, sum_f, cnt_f = stats
    
    # Tính trung bình, xử lý chia cho 0 nếu không có dữ liệu
    avg_male = sum_m / cnt_m if cnt_m > 0 else 0
    avg_female = sum_f / cnt_f if cnt_f > 0 else 0
    
    # Chuẩn bị string hiển thị
    str_male = f"{avg_male:.2f}" if cnt_m > 0 else "NA"
    str_female = f"{avg_female:.2f}" if cnt_f > 0 else "NA"
    
    return f"{title} - Male_Avg: {str_male}, Female_Avg: {str_female}"

In [15]:
# 8. Hiển thị kết quả
results = final_data.map(format_final_result).collect()

for line in results:
    print(line)

The Lord of the Rings: The Fellowship of the Ring (2001) - Male_Avg: 4.00, Female_Avg: 3.80
Sunset Boulevard (1950) - Male_Avg: 4.33, Female_Avg: 4.50
Mad Max: Fury Road (2015) - Male_Avg: 4.00, Female_Avg: 3.32
No Country for Old Men (2007) - Male_Avg: 3.92, Female_Avg: 3.83
Fight Club (1999) - Male_Avg: 3.50, Female_Avg: 3.50
The Godfather: Part II (1974) - Male_Avg: 4.06, Female_Avg: 3.94
The Social Network (2010) - Male_Avg: 4.00, Female_Avg: 3.67
Lawrence of Arabia (1962) - Male_Avg: 3.55, Female_Avg: 3.31
The Terminator (1984) - Male_Avg: 3.93, Female_Avg: 4.14
Gladiator (2000) - Male_Avg: 3.59, Female_Avg: 3.64
The Silence of the Lambs (1991) - Male_Avg: 3.33, Female_Avg: 3.00
The Lord of the Rings: The Return of the King (2003) - Male_Avg: 3.75, Female_Avg: 3.90
E.T. the Extra-Terrestrial (1982) - Male_Avg: 3.81, Female_Avg: 3.55
Psycho (1960) - Male_Avg: NA, Female_Avg: 4.00
